<a href="https://colab.research.google.com/github/pumazzo/DnCNN-tf2/blob/master/notebooks/es7/AML_2024_Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Advanced ML for Physics- aa 2023/24#
##Home Project #3##

**Scope:** Utilize a Graph Neural Network (GNN) employing PointNet++ architecture for jet tagging, the task is graph classification.

**Tasks:**
Given the dataset described in the next cell:

- Data Preparation:
    - Load the dataset provided in the subsequent cell.
    - Construct a graph representation from the point cloud data.
    - Generate the adjacency matrix based on the distances between   points in the $(\eta, \phi)$ plane, utilizing either a distance threshold or a fixed number of neighbors. Note that the number of neighbors or the distance threshold is a hyperparameter of the network.

    - For efficiency during training, consider limiting the number of particles per jet to fewer than 100 (e.g., a maximum of 30 nodes per graph).
    - Visualize a subset of events for each class to gain insights.

- Model Implementation:
    - Develop a PointNet++ model to conduct graph classification.

- Evaluation:
    - Assess the performance over the test set.
    - Present the results using a Receiver Operating Characteristic (ROC) curve and a confusion matrix.
    - Your performance evaluation will not be based on the achieved level of performance. It is adviced to  comment on your notebook, providing justifications for the decisions made and insights into the results obtained. **Clarity in reporting the results is considered more significant than the actual performance achieved**.


**NOTE:** when done, upload the notebook of the project on: [TOBEOPEN]

**Physics context:** <p>
At the extreme energies of the Large Hadron Collider, massive particles can be produced with such high Lorentz boost that their decays into hadrons (hadron jets) are collimated in such a way that the resulting particles overlap. Deducing whether the substructure of an observed jet is due to a single low-mass particle or to multiple decay objects of a high-mass particle is an important problem in LHC data analysis. Traditional approaches are based on high-level observables built from theoretical models of energy deposition in calorimeters and charged tracks parameters reconsrtcuted in the inner tracker, but the complexity of the data makes this task an excellent candidate for the application of deep learning tools. The jet constituents can be in fact represented either as 2D or 3D images,lending itself to the natural application of image classification techniques (CNN, ViT etc.), or as pointclouds/graphs, that can be classified with GNNs, or as sequences, that can be analysed by RNNs or Transformers.


**Dataset:** <p>
The dataset is the *JetDataset*: you can find a description in [this presentation](https://docs.google.com/presentation/d/1dHhPSFPlhy332SIJnmu5kwHRd0jvReKsr6ma0Cgnrac/edit?usp=sharing)


In [1]:
#imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import h5py
import glob

import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)

import torch.nn as nn
import torch.nn.functional as F

2.2.1+cu121
0.17.1+cu121


In [2]:
# dataset download
# we'll clone a github repository from M. Pierini containing the dataset
! git clone https://github.com/pierinim/tutorials.git
! ls tutorials/Data/JetDataset/

Cloning into 'tutorials'...
remote: Enumerating objects: 707, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 707 (delta 58), reused 105 (delta 35), pack-reused 579
Receiving objects: 100% (707/707), 566.43 MiB | 17.72 MiB/s, done.
Resolving deltas: 100% (260/260), done.
Updating files: 100% (79/79), done.
jetImage_7_100p_0_10000.h5	jetImage_7_100p_40000_50000.h5	jetImage_7_100p_70000_80000.h5
jetImage_7_100p_10000_20000.h5	jetImage_7_100p_50000_60000.h5	jetImage_7_100p_80000_90000.h5
jetImage_7_100p_30000_40000.h5	jetImage_7_100p_60000_70000.h5


In [3]:
# define device to use (cpu/gpu)
if torch.cuda.is_available():
  print('# of GPUs available: ', torch.cuda.device_count())
  print('First GPU type: ',torch.cuda.get_device_name(0))
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Computation device: {device}\n")

Computation device: cpu



## Data Handling

The dataset consists of a list of jets. For each jet, we have up to 100 particles (zero-padding is used in case a jet has less than 100 particles).

For each particle, we have 16 features:

* the four-momentum in cartesian coordinates $(p_x,p_y,p_z,E)$
* the energy divided by the jet energy ($E_{rel}$)
* the transverse momentum ($p_T$), i.e. the momentum projected on the plane transverse to proton beams
* the momentum transverse to the jet direction ($p_{Trel}$)
* the pseudorapidity ($\eta$), a function of the polar angle (see https://en.wikipedia.org/wiki/Pseudorapidity)
* the pseudorapidity relative to the jet direction ($\eta_{rel}$)
* the pseudorapidity relative to the jet direction ($\eta_{rot}$) after a rotation is applied so that the jet image looks vertical
* the azimuth angle ($\phi$)
* the azimuth angle relative to the jet direction ($\phi_{rel}$)
* the azimuth angle relative to the jet direction ($\phi_{rot}$) after a rotation is applied so that the jet image looks vertical

The ground truth is incorporated in a ['j_g', 'j_q', 'j_w', 'j_z', 'j_t] vector of boolean, taking the form:

* $[1, 0, 0, 0, 0]$ for gluons
* $[0, 1, 0, 0, 0]$ for quarks
* $[0, 0, 1, 0, 0]$ for W bosons (with W  qq)
* $[0, 0, 0, 1, 0]$ for Z bosons (with Z  qq)
* $[0, 0, 0, 0, 1]$ for top quarks (with t  Wq  qqq)

This is what is called 'one-hot' encoding of a discrete label (typical of ground truth for classification problems).

Additional information on this dataset and data handling can be found [HERE](https://github.com/pierinim/tutorials/blob/master/GGI_Jan2021/Lecture1/Notebook1_ExploreDataset.ipynb)

In [4]:
# data is stored in hirearchical data format (h5), a file format desgined to store and maniuplate large size data structures
# .h5 files can be accesses in python using the h5py library (https://docs.h5py.org/en/stable/)

# read dataset (only 50k events to keep training time ~20' on google colab, better performance can be obtained adding the three commented files
target = np.array([])
p_data = np.array([])
datafiles = [#'tutorials/Data/JetDataset/jetImage_7_100p_80000_90000.h5',
             #'tutorials/Data/JetDataset/jetImage_7_100p_70000_80000.h5',
             #'tutorials/Data/JetDataset/jetImage_7_100p_60000_70000.h5',
             #'tutorials/Data/JetDataset/jetImage_7_100p_50000_60000.h5',
             #'tutorials/Data/JetDataset/jetImage_7_100p_40000_50000.h5',
             #'tutorials/Data/JetDataset/jetImage_7_100p_30000_40000.h5',
             'tutorials/Data/JetDataset/jetImage_7_100p_10000_20000.h5',
             'tutorials/Data/JetDataset/jetImage_7_100p_0_10000.h5']


# let's print what is contained in one of the files:
f = h5py.File(datafiles[0])
print(list(f.keys()))
f.close()

# each file contains different numpy arrays, the ones we are interested in are:
# "jetConstituentsList": containing for each jet, and for each jet constituent particle (up to 100 particles) the 16 features associated to the jet particle
# "jets": containing for each jet several (59) global features of the jet, we are here interested in the elements from -6:-1 that provide a onehot encoding of the jet-type label ['j_g', 'j_q', 'j_w', 'j_z', 'j_t]


#loop over the files and concatenate the content to the p_adat and target arrays
for fileIN in datafiles:
    print("Appending %s" %fileIN)
    f = h5py.File(fileIN) #read the h5 file
    data = np.array(f.get("jetConstituentList")) #jet constituents
    targ = np.array(f.get('jets')[0:,-6:-1])  #select ['j_g', 'j_q', 'j_w', 'j_z', 'j_t] out of the 59 features presents in the container
    p_data = np.concatenate([p_data, data], axis=0) if p_data.size else data
    target = np.concatenate([target, targ], axis=0) if target.size else targ
    del data, targ
    f.close()

print(target.shape, p_data.shape)

p_featurenames = ['j1_px','j1_py','j1_pz','j1_e','j1_erel','j1_pt','j1_ptrel',
 'j1_eta','j1_etarel','j1_etarot','j1_phi','j1_phirel','j1_phirot',
 'j1_deltaR','j1_costheta','j1_costhetarel']

print(p_featurenames)

['jetConstituentList', 'jetFeatureNames', 'jetImage', 'jetImageECAL', 'jetImageHCAL', 'jets', 'particleFeatureNames']
Appending tutorials/Data/JetDataset/jetImage_7_100p_50000_60000.h5
Appending tutorials/Data/JetDataset/jetImage_7_100p_40000_50000.h5
Appending tutorials/Data/JetDataset/jetImage_7_100p_30000_40000.h5
Appending tutorials/Data/JetDataset/jetImage_7_100p_10000_20000.h5
Appending tutorials/Data/JetDataset/jetImage_7_100p_0_10000.h5
(50000, 5) (50000, 100, 16)
['j1_px', 'j1_py', 'j1_pz', 'j1_e', 'j1_erel', 'j1_pt', 'j1_ptrel', 'j1_eta', 'j1_etarel', 'j1_etarot', 'j1_phi', 'j1_phirel', 'j1_phirot', 'j1_deltaR', 'j1_costheta', 'j1_costhetarel']
